## 0.0 Imports

In [246]:
import pandas as pd
import re
import emoji
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np
from collections import Counter


## 1. Base de dados e Limpeza

A base de dados foi extraída do Kaggl

In [247]:
df = pd.read_csv('data.csv')
df

,id,tweet_text,tweet_date,sentiment,query_used
0,1031761728445530112,@Tixaa23 14 para eu ir :),Tue Aug 21 04:35:39 +0000 2018,Positivo,:)
1,1031761040462278656,@drexalvarez O meu like eu já dei na época :),Tue Aug 21 04:32:55 +0000 2018,Positivo,:)
2,1031760962372689920,Eu só queria conseguir comer alguma coisa pra ...,Tue Aug 21 04:32:37 +0000 2018,Positivo,:)
3,1031760948250456066,:D que lindo dia !,Tue Aug 21 04:32:33 +0000 2018,Positivo,:)
4,1031760895985246208,"@Primo_Resmungao Pq da pr jeito!!é uma ""oferta...",Tue Aug 21 04:32:21 +0000 2018,Positivo,:)
...,...,...,...,...,...
785809,1050705141207367680,Acordar 8 horas é tão bom :),Fri Oct 12 11:10:01 +0000 2018,Positivo,:)
785810,1050706655049109504,"@mayckcunha Olá, Mayck. Você já é cliente Clar...",Fri Oct 12 11:16:02 +0000 2018,Positivo,:)
785811,1050705846907392005,Opa tava na merda mm e fiquei logo mais feliz ...,Fri Oct 12 11:12:49 +0000 2018,Positivo,:)
785812,1050705490232127489,@andrebraga2806 Foi como a tua lealdade :),Fri Oct 12 11:11:24 +0000 2018,Positivo,:)


In [248]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 785814 entries, 0 to 785813
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id          785814 non-null  int64 
 1   tweet_text  785814 non-null  object
 2   tweet_date  785814 non-null  object
 3   sentiment   785814 non-null  object
 4   query_used  785814 non-null  object
dtypes: int64(1), object(4)
memory usage: 30.0+ MB


### 1.1. Diminuição da base

In [249]:
#Devido a base ter um tamanho muito grande (785814 instancias) e necessitar de muito tempo e memória para processamento, a base será diminuída a 1000 instancias, sendo 50% como positiva e 50% negativa

# filtrar entre positivos e negativos
positive = df[df['sentiment'] == 'Positivo']
negative = df[df['sentiment'] == 'Negativo']

# 500 instancias/linhas aleatórias de cada sentimento
positive = positive.sample(n=2000, random_state=42)
negative = negative.sample(n=2000, random_state=42)

# juntar e embaralhar
df = pd.concat([positive, negative])
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
df

,id,tweet_text,tweet_date,sentiment,query_used
0,1047352136202424320,o linux entrou :) depois que ele entra ele sim...,Wed Oct 03 05:06:22 +0000 2018,Positivo,:)
1,1031657254594981888,eu tenho a síndrome da existência sempre acho ...,Mon Aug 20 21:40:31 +0000 2018,Negativo,:(
2,1045447141882318853,oi apoiem meu video :)) https://t.co/ODjuL9UHLp,Thu Sep 27 22:56:36 +0000 2018,Positivo,:)
3,1031991144748601344,o brasil é complicadissimo :( https://t.co/CVS...,Tue Aug 21 19:47:17 +0000 2018,Negativo,:(
4,1042181437947109376,"@S94GK Não... Já melhorei, bff. :(",Tue Sep 18 22:39:52 +0000 2018,Negativo,:(
...,...,...,...,...,...
3995,1045315889313136640,"Dólar caiu :) - R$4,00 às 11:10",Thu Sep 27 14:15:03 +0000 2018,Positivo,:)
3996,1037332731364089857,@kcclxser oi sdv :),Wed Sep 05 13:32:50 +0000 2018,Positivo,:)
3997,1031489543567036418,minha mãe faz questão de me irritar logo de ma...,Mon Aug 20 10:34:05 +0000 2018,Positivo,:)
3998,1030831061612339202,Pleno sábado e eu não sei o que fazer :(,Sat Aug 18 14:57:31 +0000 2018,Negativo,:(


### 1.2. Limpeza

In [250]:
# excluir colunas "inuteis"
df = df.drop(columns=['id', 'tweet_date', 'query_used'], axis=True)
df. head()

,tweet_text,sentiment
0,o linux entrou :) depois que ele entra ele sim...,Positivo
1,eu tenho a síndrome da existência sempre acho ...,Negativo
2,oi apoiem meu video :)) https://t.co/ODjuL9UHLp,Positivo
3,o brasil é complicadissimo :( https://t.co/CVS...,Negativo
4,"@S94GK Não... Já melhorei, bff. :(",Negativo


In [251]:
# funções para limpeza de texto

def remove_emoji(text):
    return ''.join(char for char in text if not emoji.is_emoji(char))

def remove_emoticons(text):
    emoticon_regex = re.compile(r'[:;=][\)\(\[dDpP]|[\)\(\[dDpP][:;=]|<3|:\'\(|:\'\)|:o|:O|:\|')
    return emoticon_regex.sub('', text)

def remove_links(text):
    link_regex = re.compile(r'http\S+|www\S+')
    return link_regex.sub('', text)

def remove_usernames(text):
    return re.sub(r'@\w+', '', text)

def remove_special_chars(text):
    special_chars_regex = re.compile(r'[^A-Za-z0-9\s]')
    return special_chars_regex.sub('', text)

def clean_text(text):
    text = remove_emoji(text)
    text = remove_emoticons(text)
    text = remove_links(text)
    text = remove_usernames(text)
    text = remove_special_chars(text)
    text = text.lower()
    
    return text

df['tweet_text'] = df['tweet_text'].apply(clean_text)
df.head()


,tweet_text,sentiment
0,o linux entrou depois que ele entra ele simpl...,Positivo
1,eu tenho a sndrome da existncia sempre acho q ...,Negativo
2,oi apoiem meu video,Positivo
3,o brasil complicadissimo,Negativo
4,no j melhorei bff,Negativo


### 1.3 Tokenização

In [252]:
def preprocess_text(text):
    # Tokenizar o texto
    tokens = word_tokenize(text, language='portuguese')
    # Remove stopwords
    stop_words = set(stopwords.words('portuguese'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(filtered_tokens)

df['tweet_text'] = df['tweet_text'].apply(preprocess_text)

## 2. Separação de treino e teste

In [253]:
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline


# Separar os dados em variáveis independentes (X) e dependentes (y) (features e target)
X = df['tweet_text']
y = df['sentiment']

# configuração de k=10
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)


## 3. Modelos e métrica de desempenho

### 3.1. Funções acessórios

In [254]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.pipeline import Pipeline


# função que roda o modelo e calcula métricas
def evaluate_model(model, X, y, cv):
    
    scoring = {
        'accuracy': 'accuracy',
        'precision': 'precision_weighted',
        'recall': 'recall_weighted',
        'f1': 'f1_weighted',
        'error': make_scorer(lambda y_true, y_pred: 1 - accuracy_score(y_true, y_pred))  # Cálculo do erro como 1 - acurácia
    }

    # validação cruzada
    results = cross_validate(model, X, y, cv=cv, scoring=scoring)

    # métricas de desempenho
    metrics = {
        'Acurácia': results['test_accuracy'].mean(),
        'Precisão': results['test_precision'].mean(),
        'Recall': results['test_recall'].mean(),
        'F1 Score': results['test_f1'].mean(),
        'Erro': results['test_error'].mean()
    }

    metrics_df = pd.DataFrame([metrics]).T
    return metrics_df

### 3.2. Regressão Logística

In [255]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler


reg_pipeline = make_pipeline(
    CountVectorizer(),
    StandardScaler(with_mean=False), 
    LogisticRegression(max_iter=2000)
)

metrics_df = evaluate_model(reg_pipeline, X, y, cv)
print(metrics_df)

                 0
Acurácia  0.640500
Precisão  0.642793
Recall    0.640500
F1 Score  0.639010
Erro      0.359500


### 3.3. Naive Bayes

In [256]:
from sklearn.naive_bayes import MultinomialNB

nb_pipeline = Pipeline([
    ('vect', CountVectorizer()),  #vetoriza os dados
    ('clf', MultinomialNB())     
])

metrics_df = evaluate_model(nb_pipeline, X, y, cv)
print(metrics_df)

                 0
Acurácia  0.699000
Precisão  0.705816
Recall    0.699000
F1 Score  0.696447
Erro      0.301000


### 3.4. SVM

In [257]:
from sklearn.svm import SVC

svm_pipeline = Pipeline([
    ('vect', CountVectorizer()),   #vetoriza os dados
    ('clf', SVC(kernel='linear'))    
])

metrics_df = evaluate_model(svm_pipeline, X, y, cv)
print(metrics_df)

                 0
Acurácia  0.674500
Precisão  0.676101
Recall    0.674500
F1 Score  0.673676
Erro      0.325500


### 3.5. Árvore de Decisão

In [258]:
from sklearn.tree import DecisionTreeClassifier

dt_pipeline = Pipeline([
    ('vect', CountVectorizer()),  #vetoriza os dados
    ('clf', DecisionTreeClassifier())      
])

metrics_df = evaluate_model(dt_pipeline, X, y, cv)
print(metrics_df)

                 0
Acurácia  0.624000
Precisão  0.630853
Recall    0.624000
F1 Score  0.618507
Erro      0.376000


### 3.6. Random Forest

In [259]:
from sklearn.ensemble import RandomForestClassifier

rf_pipeline = Pipeline([
    ('vect', CountVectorizer()), #vetoriza os dados
    ('clf', RandomForestClassifier(n_estimators=100, random_state=42))      #aplica algoritmo
])

metrics_df = evaluate_model(rf_pipeline, X, y, cv)
print(metrics_df)

                 0
Acurácia  0.679000
Precisão  0.685971
Recall    0.679000
F1 Score  0.675627
Erro      0.321000


### 3.7. KNN

In [261]:

from sklearn.base import BaseEstimator, ClassifierMixin
from collections import Counter
import numpy as np

# cria um classificador KNN
class CustomKNNClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, k=3):
        # define o numero de vizinhos
        self.k = k
    
    def fit(self, X, y):
        self.X_train = X
        self.y_train = y if isinstance(y, np.ndarray) else y.values
        self.classes_ = np.unique(self.y_train)
        return self
    
    def euclidean_distance(self, x1, x2):
        # calcula a distância Euclidiana e converte forem matrizes esparsas em arrays densos
        x1_dense = x1.toarray() if hasattr(x1, 'toarray') else x1
        x2_dense = x2.toarray() if hasattr(x2, 'toarray') else x2
        return np.sqrt(np.sum((x1_dense - x2_dense) ** 2))
    
    def predict(self, X):
        predictions = []
        for test_point in X:
            # Calcula a distância entre teste e treino
            distances = [self.euclidean_distance(test_point, x) for x in self.X_train]
            k_indices = np.argsort(distances)[:self.k]
            k_nearest_labels = [self.y_train[i] for i in k_indices]
            most_common = Counter(k_nearest_labels).most_common(1)
            predictions.append(most_common[0][0])
        return np.array(predictions)  # Retorna as previsões como um array

knn_pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', CustomKNNClassifier(k=3))
])

metrics_df = evaluate_model(knn_pipeline, X, y, cv)
metrics_df

,0
Acurácia,0.580000
Precisão,0.587176
Recall,0.580000
F1 Score,0.570712
Erro,0.420000


### 3.8. Comitê

In [265]:
def weights(acuracias):
    total_acuracia = sum(acuracias)
    pesos = [acuracia / total_acuracia for acuracia in acuracias]
    return pesos

acuracias = [0.699000, 0.624000, 0.679000, 0.640500, 0.674500, 0.580000]  # Acurácias dos classificadores
weights = weights(acuracias)
print("Pesos calculados:", weights)

Pesos calculados: [0.17936874518860663, 0.1601231716705158, 0.17423659225044907, 0.16435719784449576, 0.1730818578393636, 0.14883243520656916]


In [266]:
from sklearn.ensemble import VotingClassifier

# rodar o comitê com todos os algoritmos atribuindo pesos pra cada algoritmo
ensemble_model = VotingClassifier(estimators=[
    ('gnb', nb_pipeline),
    ('dt', dt_pipeline),
    ('rf', rf_pipeline),
    ('lr', reg_pipeline),
    ('svm', svm_pipeline),
    ('knn', knn_pipeline)
],    voting='hard', 
    weights=pesos)  


metrics_df = evaluate_model(ensemble_model, X, y, cv)
metrics_df

                 0
Acurácia  0.694250
Precisão  0.699489
Recall    0.694250
F1 Score  0.691950
Erro      0.305750


In [271]:
# criando visão em tabela
dados = {
    'Algoritmo': ['Naive Bayes', 'SVM', 'Árvore de Decisão', 'Random Forest', 'Regressão Logística', 'KNN', 'Comitê'],
    'Acurácia': [0.699, 0.6745, 0.624, 0.679, 0.6405, 0.58, 0.694],
    'Precisão': [0.705816, 0.676101, 0.630853, 0.685971, 0.642793, 0.587176, 0.699],
    'Recall': [0.699, 0.6745, 0.624, 0.679, 0.6405, 0.58, 0.694],
    'F1 Score': [0.696447, 0.673676, 0.618507, 0.675627, 0.63901, 0.570712, 0.691],
    'Erro': [0.301, 0.3255, 0.376, 0.321, 0.3595, 0.42, 0.305]
}

table = pd.DataFrame(dados)
table

,Algoritmo,Acurácia,Precisão,Recall,F1 Score,Erro
0,Naive Bayes,0.6990,0.705816,0.6990,0.696447,0.3010
1,SVM,0.6745,0.676101,0.6745,0.673676,0.3255
2,Árvore de Decisão,0.6240,0.630853,0.6240,0.618507,0.3760
3,Random Forest,0.6790,0.685971,0.6790,0.675627,0.3210
4,Regressão Logística,0.6405,0.642793,0.6405,0.639010,0.3595
5,KNN,0.5800,0.587176,0.5800,0.570712,0.4200
6,Comitê,0.6940,0.699000,0.6940,0.691000,0.3050
